# Sign Language Digits Classifier
Using MobileNet transfer learning to classify ASL digits (0-9)

## 1. Import Dependencies

In [263]:
# Import dependencies
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## 2. Set Project Constants

In [264]:
# Project constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
VALIDATION_SPLIT = 0.2
NUM_CLASSES = 10

# Paths
DATASET_PATH = 'Sign-Language-Digits-Dataset/Dataset'
SAMPLE_PATH = 'Sign-Language-Digits-Dataset/sample'
EVAL_PATH = 'Sign-Language-Digits-Dataset/eval'

## 3. Create Dataset Splits
Creating sample/ folder (250 images for training) and eval/ folder (250 images for evaluation)

In [265]:
# Create sample/ and eval/ folders with random images
def create_dataset_split(source_path, dest_path, num_images_per_class=25, exclude_images=None):
    """Create a dataset folder with random images from source"""
    if exclude_images is None:
        exclude_images = set()
    
    # Remove destination if it exists
    if os.path.exists(dest_path):
        shutil.rmtree(dest_path)
    os.makedirs(dest_path)
    
    selected_images = set()
    
    # Process each digit (0-9)
    for digit in range(10):
        digit_source = os.path.join(source_path, str(digit))
        digit_dest = os.path.join(dest_path, str(digit))
        os.makedirs(digit_dest)
        
        # Get all images for this digit
        all_images = [f for f in os.listdir(digit_source) if f.endswith('.JPG')]
        
        # Filter out excluded images
        available_images = [img for img in all_images 
                          if os.path.join(str(digit), img) not in exclude_images]
        
        # Randomly select images
        selected = random.sample(available_images, num_images_per_class)
        
        # Copy selected images
        for img in selected:
            src = os.path.join(digit_source, img)
            dst = os.path.join(digit_dest, img)
            shutil.copy2(src, dst)
            selected_images.add(os.path.join(str(digit), img))
    
    return selected_images

# Create sample/ folder (25 images per digit)
print('Creating sample/ folder...')
sample_images = create_dataset_split(DATASET_PATH, SAMPLE_PATH, num_images_per_class=25)
print(f'Created {len(sample_images)} images in sample/ folder')

# Create eval/ folder (25 different images per digit)
print('\nCreating eval/ folder...')
eval_images = create_dataset_split(DATASET_PATH, EVAL_PATH, num_images_per_class=25, 
                                   exclude_images=sample_images)
print(f'Created {len(eval_images)} images in eval/ folder')

Creating sample/ folder...
Created 250 images in sample/ folder

Creating eval/ folder...
Created 250 images in eval/ folder


## 4. Build Model Architecture
Using MobileNetV2 as base model with frozen weights, adding custom classification head

In [266]:
# Import MobileNet and create model
print('Loading MobileNetV2...')
base_model = MobileNetV2(
    # input_shape=(100, 100, 3), 
    include_top=False, 
    weights='imagenet'
)

# Freeze base model layers
# base_model.trainable = False

# Get last 7 layers with more details
for layer in base_model.layers[-7:]:
    print(f"Layer: {layer.name}")
    print(f"  Type: {layer.__class__.__name__}")
    print(f"  Output Shape: {layer.output.shape}")
    print(f"  Params: {layer.count_params()}")
    print("-" * 50)

Loading MobileNetV2...


/tmp/ipykernel_37738/1689098958.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Layer: block_16_depthwise_BN
  Type: BatchNormalization
  Output Shape: (None, None, None, 960)
  Params: 3840
--------------------------------------------------
Layer: block_16_depthwise_relu
  Type: ReLU
  Output Shape: (None, None, None, 960)
  Params: 0
--------------------------------------------------
Layer: block_16_project
  Type: Conv2D
  Output Shape: (None, None, None, 320)
  Params: 307200
--------------------------------------------------
Layer: block_16_project_BN
  Type: BatchNormalization
  Output Shape: (None, None, None, 320)
  Params: 1280
--------------------------------------------------
Layer: Conv_1
  Type: Conv2D
  Output Shape: (None, None, None, 1280)
  Params: 409600
--------------------------------------------------
Layer: Conv_1_bn
  Type: BatchNormalization
  Output Shape: (None, None, None, 1280)
  Params: 5120
--------------------------------------------------
Layer: out_relu
  Type: ReLU
  Output Shape: (None, None, None, 1280)
  Params: 0
-------------

In [267]:
# Build model on top of MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
predictions = Dense(
    NUM_CLASSES,
    activation='softmax',
    # kernel_regularizer=regularizers.l2(0.1)
)(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers[:-7]:
    layer.trainable = False

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print('\nModel architecture:')
# Get last 7 layers with more details
for layer in model.layers[-7:]:
    print(f"Layer: {layer.name}")
    print(f"  Type: {layer.__class__.__name__}")
    print(f"  Output Shape: {layer.output.shape}")
    print(f"  Params: {layer.count_params()}")
    print(f" Trainable: {layer.trainable}")
    print("-" * 50)


Model architecture:
Layer: block_16_project_BN
  Type: BatchNormalization
  Output Shape: (None, None, None, 320)
  Params: 1280
 Trainable: True
--------------------------------------------------
Layer: Conv_1
  Type: Conv2D
  Output Shape: (None, None, None, 1280)
  Params: 409600
 Trainable: True
--------------------------------------------------
Layer: Conv_1_bn
  Type: BatchNormalization
  Output Shape: (None, None, None, 1280)
  Params: 5120
 Trainable: True
--------------------------------------------------
Layer: out_relu
  Type: ReLU
  Output Shape: (None, None, None, 1280)
  Params: 0
 Trainable: True
--------------------------------------------------
Layer: global_average_pooling2d_55
  Type: GlobalAveragePooling2D
  Output Shape: (None, 1280)
  Params: 0
 Trainable: True
--------------------------------------------------
Layer: dropout_14
  Type: Dropout
  Output Shape: (None, 1280)
  Params: 0
 Trainable: True
--------------------------------------------------
Layer: dens

## 5. Create Data Generators
Setting up training and validation generators with 80/20 split

In [268]:
# Create image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT,
    # Data augmentation parameters
    rotation_range=15,              # Randomly rotate images by up to 15 degrees
    zoom_range=0.15,                # Randomly zoom in/out by up to 15%
    brightness_range=[0.8, 1.2],    # Randomly adjust brightness
    fill_mode='nearest'             # Fill pixels after transformations
)

# Validation generator (no augmentation, only rescaling)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Validation generator
validation_generator = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

print(f'\nTraining samples: {train_generator.samples}')
print(f'Validation samples: {validation_generator.samples}')

Found 1653 images belonging to 10 classes.
Found 409 images belonging to 10 classes.

Training samples: 1653
Validation samples: 409


## 6. Train Model

In [269]:
# Train the model
print('Training model...')
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1
)

Training model...
Epoch 1/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 36s 649ms/step - accuracy: 0.4870 - loss: 1.5191 - val_accuracy: 0.5819 - val_loss: 1.2674
Epoch 2/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 35s 662ms/step - accuracy: 0.7792 - loss: 0.7021 - val_accuracy: 0.6039 - val_loss: 1.0714
Epoch 3/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 41s 663ms/step - accuracy: 0.8433 - loss: 0.5092 - val_accuracy: 0.6528 - val_loss: 0.9570
Epoch 4/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 33s 627ms/step - accuracy: 0.8730 - loss: 0.4281 - val_accuracy: 0.6748 - val_loss: 0.9081
Epoch 5/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 32s 607ms/step - accuracy: 0.8857 - loss: 0.3717 - val_accuracy: 0.6968 - val_loss: 0.8342
Epoch 6/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 32s 617ms/step - accuracy: 0.9093 - loss: 0.3197 - val_accuracy: 0.7311 - val_loss: 0.7576
Epoch 7/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 32s 611ms/step - accuracy: 0.9189 - loss: 0.2815 - val_accuracy: 0.7335 - val_loss: 0.7255
Epoch 8/25
52/52 ━━━━━━━━━━━━━━━━━━━━ 31s 600ms/step - accuracy: 0.9195 - loss:

## 7. Visualize Training History

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 8. Evaluate Model
Testing on unseen eval dataset

In [ ]:
# Evaluate model on eval/ folder
eval_datagen = ImageDataGenerator(rescale=1./255)

eval_generator = eval_datagen.flow_from_directory(
    EVAL_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print('Evaluating model on eval dataset...')
eval_loss, eval_accuracy = model.evaluate(eval_generator)
print(f'\nEval Loss: {eval_loss:.4f}')
print(f'Eval Accuracy: {eval_accuracy:.4f}')

## 9. Confusion Matrix & Classification Report

In [ ]:
# Generate predictions for confusion matrix
eval_generator.reset()
predictions = model.predict(eval_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = eval_generator.classes

# Create confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.title('Confusion Matrix - Sign Language Digits')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Print classification report
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, 
                          target_names=[str(i) for i in range(10)]))